In [ ]:
#TASK 5
# HANDWRITTEN TEXT GENERATION

In [14]:
import pandas as pd
import random
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.callbacks import LambdaCallback

In [2]:
# Load and preprocess the dataset
data = pd.read_csv('E:/Dhruvi Desktop Files/Internship and Course Certificate/PDF_ML_CodSoft_ML/train_essays_v1.csv')
data

,text,label,prompt_name,source,RDizzl3_seven
0,The Electoral College is a complex system that...,1,Does the electoral college work?,gemini_pro,True
1,"The exploration of Venus, Earth's closest plan...",1,Exploring Venus,gemini_pro,True
2,## Does the Electoral College Work?\n\nThe Ele...,1,Does the electoral college work?,gemini_pro,True
3,"In the vast realm of literature, there exists ...",1,"""A Cowboy Who Rode the Waves""",gemini_pro,True
4,"In the realm of modern transportation, the adv...",1,Driverless cars,gemini_pro,True
...,...,...,...,...,...
3495,"The Electoral College, a distinctive feature o...",1,Does the electoral college work?,gemini_pro,True
3496,The Electoral College is a system of indirect ...,1,Does the electoral college work?,gemini_pro,True
3497,"The electoral College, an integral part of the...",1,Does the electoral college work?,gemini_pro,True
3498,"The Electoral College, a distinctive feature o...",1,Does the electoral college work?,gemini_pro,True


In [5]:
text = ''.join(data['text'].astype(str).tolist())[:1000]  # Limit to first 1 million characters for example
text

'The Electoral College is a complex system that has been in place since the founding of the United States. It is a system that has both advantages and disadvantages. On the one hand, it ensures that every state has a say in who becomes President, regardless of population. On the other hand, it can lead to situations where a candidate who wins the popular vote does not win the Presidency, as happened in 2016.\n\nThere are a number of arguments in favor of the Electoral College. One argument is that it helps to protect the rights of minority groups. In a direct popular vote, it is possible for a candidate to win by a landslide in a few large states, while losing by narrow margins in many smaller states. This could lead to a situation where the President is elected by a minority of the population. The Electoral College, on the other hand, ensures that every state has a say in the election, regardless of its size.\n\nAnother argument in favor of the Electoral College is that it encourages 

In [6]:
# Create character mappings
chars = sorted(list(set(text)))
char_indices = {char: i for i, char in enumerate(chars)}
indices_char = {i: char for i, char in enumerate(chars)}

In [7]:
# Length of extracted character sequences
maxlen = 40
step = 3

# Generate sequences and their corresponding next characters
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [12]:
# Vectorize sequences into input and output
import tensorflow as tf
x = []
y = []
for i, sentence in enumerate(sentences):
    x.append([char_indices[char] for char in sentence])
    y.append(char_indices[next_chars[i]])

# Convert x and y to TensorFlow/Keras compatible format
x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=maxlen, padding='pre')
y = tf.keras.utils.to_categorical(y, num_classes=len(chars))


In [15]:
# Build the RNN model
model = Sequential([
    Embedding(len(chars), 50, input_length=maxlen),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

C:\Users\DHRUVI\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
# Function to sample next character based on the model's prediction
def sample(preds, temperature=1.0):
    preds = tf.squeeze(preds).numpy()
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
# Function to generate text using the trained model
def generate_text(seed_text, length=400, temperature=1.0):
    generated = seed_text
    sys.stdout.write(generated)
    
    for _ in range(length):
        x_pred = [[char_indices[char] for char in seed_text]]
        x_pred = tf.keras.preprocessing.sequence.pad_sequences(x_pred, maxlen=maxlen, padding='pre')
        
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        
        seed_text = seed_text[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [20]:
# Callback to generate text at the end of each epoch
import numpy as np
def on_epoch_end(epoch, _):
    print(f'\nGenerating text after epoch {epoch + 1}:')
    start_index = random.randint(0, len(text) - maxlen - 1)
    seed_text = text[start_index: start_index + maxlen]
    generate_text(seed_text)

# Train the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
print_callback

In [22]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=[print_callback])

Epoch 1/5
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 3.2525
Generating text after epoch 1:
s that every state has a say in who becoo 1ao fatzn yoSetaoetotettAs a ttaztotCaaototso eten erCa  egd gtgt i aa O e tC adtz a oo z erh ttan. et o aer o1     tidvtc0nli ionag tt Claa  t1  tmAam c t astgt et n   ea lo,t w  Ce oxv   e ml attaxa  eC Ur a t  1 tte St U  yuzs a tal aeta.y itpTtnenseCpxs 2tsmiit cto  ,ht  ee  tgIyatIez e mtaE alt ttl gA g o6fdogt0 yasdxc tarta auen  aSa md atra l g nbe2calaCho o y0taot  deiraett yoteayetp.tta
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 12s/step - loss: 3.2199
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 3.0378
Generating text after epoch 2:
ion, regardless of its size.

Another ar b vd ee   fny xo  eUaaoEtea tena t ezay i r fmtrt.en n gt.ewa nttnpen  e e tzcty ai  vandodnaleCoieorhig
 vtcae ,edCaeced e  daea e  abeeeatebeneean0ocaat theo   ei  eneaax Ae tts og  ao et  tar te yaeSerdlao   sa g  t e,tat , n Pnvyt xn  a  rttef g  tte  matt p oa eanAcme   a

In [23]:
# Generate text after training
print('\nGenerating final text:')
start_index = random.randint(0, len(text) - maxlen - 1)
start_index


Generating final text:


850

In [24]:
seed_text = text[start_index: start_index + maxlen]
generate_text(seed_text)

es that every state has a say in the eleUe1gcrpeetatr slr.saes iaoysl cw  i vd gr nt stEabeeen irrtodrytdrl1tt E   mi e .alitw s  soihiiot   cbooTrlcae .st r i gliisn rfrsyai agUgTvgas6animsslsrdrwg.taydUrapa ansige ggweh.igA.t.eareg ae treiwenm swisne,dtegdo eay  lfsgdeahen gshhe atsoeglgtotnstIes6
 iheocgsh ePtmovat EvgdnC ohinsni    s6aeoesnisyizi nteysettsnggealsalUiclotetna xa  igtt   relsEar arPoniit bstc1anat s dieUafmyiaw tc glS
